# Models

# `group-assignment-model.ipynb`

In [1]:
import gurobipy as gp
from gurobipy import GRB

---
## `Google Collab Zane's License`
---

In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package
from gurobipy import GRB

params = {
"WLSACCESSID": '00afa35c-1548-45ee-88a2-4f443037364f',
"WLSSECRET": 'db8205a2-7aea-478a-b4b3-079b14c81803',
"LICENSEID": 2376633,
}
env = gp.Env(params=params)

---
## `Dummy Data`
---

In [3]:
import pandas as pd
import math

banker = pd.read_excel('.//banker_data//banker_data.xlsx')
bank = pd.read_csv('.//bank_loan_data//bank.csv')
loan = pd.read_csv('.//bank_loan_data//loan.csv')
loan_type = loan['loan_type'].unique()   # loan_type = [0,1]
# type = ["living", "investment"]

P = range(6)
B = range(3)
L = range(15)
T = range(len(loan_type))
print([P,B,L,T])

banker = banker.iloc[0:6]
bank = bank.iloc[0:3]
loan = loan.iloc[0:15]

banker.loc[0, "capacity"] = 750
banker.loc[1,"capacity"] = 250
banker.loc[2, "capacity"] = 500
banker.loc[3,"capacity"] = 250
banker.loc[4, "capacity"] = 750
banker.loc[5,"capacity"] = 1000
# banker.loc[6, "capacity"] = 500
# banker.loc[7,"capacity"] = 250
# banker.loc[8, "capacity"] = 500
# banker.loc[9,"capacity"] = 500

bank.loc[0,"capacity"] = 1000
bank.loc[1,"capacity"] = 5000
bank.loc[2,"capacity"] = 3500

loan.loc[0,"loan_amount"] = 300
loan.loc[0,"loan_type"] = 1
loan.loc[1,"loan_amount"] = 200
loan.loc[1,"loan_type"] = 0
loan.loc[2,"loan_amount"] = 400
loan.loc[2,"loan_type"] = 0
loan.loc[3,"loan_amount"] = 100
loan.loc[3,"loan_type"] = 0
loan.loc[4,"loan_amount"] = 300
loan.loc[4,"loan_type"] = 1
loan.loc[5,"loan_amount"] = 200
loan.loc[5,"loan_type"] = 0
loan.loc[6,"loan_amount"] = 400
loan.loc[6,"loan_type"] = 0
loan.loc[7,"loan_amount"] = 100
loan.loc[7,"loan_type"] = 0
loan.loc[8,"loan_amount"] = 400
loan.loc[8,"loan_type"] = 0
loan.loc[9,"loan_amount"] = 100
loan.loc[9,"loan_type"] = 0
loan.loc[10,"loan_amount"] = 150
loan.loc[10,"loan_type"] = 1
loan.loc[11,"loan_amount"] = 200
loan.loc[11,"loan_type"] = 0
loan.loc[12,"loan_amount"] = 600
loan.loc[12,"loan_type"] = 0
loan.loc[13,"loan_amount"] = 100
loan.loc[13,"loan_type"] = 1
loan.loc[14,"loan_amount"] = 300
loan.loc[14,"loan_type"] = 0
# loan.loc[15,"loan_amount"] = 200
# loan.loc[15,"loan_type"] = 0
# loan.loc[16,"loan_amount"] = 400
# loan.loc[16,"loan_type"] = 0
# loan.loc[17,"loan_amount"] = 100
# loan.loc[17,"loan_type"] = 0
# loan.loc[18,"loan_amount"] = 400
# loan.loc[18,"loan_type"] = 0
# loan.loc[19,"loan_amount"] = 200
# loan.loc[19,"loan_type"] = 1

        
# Dummy distance between banker and branch
dist = [[5,8,1], [6,9,2], [1,2,4], [1,1,1], [4,3,5], [6,2,4]]#, [7,1,11], [8,2,3], [6,2,1], [2,3,2]]

print("bank\n", bank.head())
print("loan\n",loan.head())
print("banker\n",banker.head())

[range(0, 6), range(0, 3), range(0, 15), range(0, 2)]
bank
                                             location        long        lat  \
0  Westfield Fountain Gate, 352 Princes Highway, ...  145.304441 -38.018206   
1               330 Collins St, Melbourne, VIC, 3000  144.963217 -37.816309   
2               460 Collins St, Melbourne, VIC, 3000  144.958852 -37.817673   

   capacity  
0    1000.0  
1    5000.0  
2    3500.0  
loan
    loan_amount  loan_type
0        300.0          1
1        200.0          0
2        400.0          0
3        100.0          0
4        300.0          1
banker
    banker_id   Latitude   Longitude  tenure (in years)  capacity  v1_rand_0_1  \
0          1 -37.695528  143.951218           8.183687       750     0.863803   
1          2 -37.551700  144.405921           8.184793       250     0.539807   
2          3 -37.537275  144.841580           6.404775       500     0.934031   
3          4 -37.563307  144.933624           4.616158       250     0.88

---
## `Full Data`
---

In [ ]:
import pandas as pd
import math

banker = pd.read_excel('.//banker_data//banker_data.xlsx')
bank = pd.read_csv('.//bank_loan_data//bank.csv')
loan = pd.read_csv('.//bank_loan_data//loan.csv')
loan_type = loan['loan_type'].unique()   # loan_type = [0,1] 
# type = ["living", "investment"]

banker['salary (in k)'] = [i*1000 for i in banker['salary (in k)']]
banker['capacity'] = [i*1000000 for i in banker['capacity']]

P = range(len(banker))
B = range(len(bank))
L = range(len(loan))
T = range(len(loan_type))
print([P,B,L,T])
        
# yoinked from ass1 
dist = [ [0]*len(B) for i in P]
for p in P:
     for b in B:
        # LA,LONG for banker is column 1,2; for banks LONG,LA is 1:2
        dist[p][b] = math.dist(banker.iloc[p, 1:3],bank.iloc[b, 2:0:-1])


print("bank\n", bank.head())
print("loan\n",loan.head())
print("banker\n",banker.head())

---

## `Model Constants` 

---

In [5]:
salary_rescale = 0.1 

---

## `Model 1`

---

In [4]:
m = gp.Model()
m.Params.OutputFlag = 1

M = 100000
x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)
 
# Objective
m.setObjective( gp.quicksum( dist[p][b]*y[p,b] for b in B for p in P) +
                gp.quicksum( banker.iloc[p]["salary (in k)"]*y[p,b]*salary_rescale for p in P for b in B),
               GRB.MINIMIZE )

## Constraints ##
m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
               <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
             name='1.each loan <= 1 bank')

m.addConstrs( (gp.quicksum( z[t,l,p] for p in P) 
               <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
             name='2.each loan to <= 1 banker')

m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
               <= 1 for p in P),
             name='3.each banker <= 1 bank')

m.addConstrs( (gp.quicksum( z[t,l,p] * loan.iloc[l]["loan_amount"] 
                           for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <=  banker.iloc[p]["capacity"] for p in P ),
             name='4.value of loans respect banker cap')

m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                           for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <= bank.iloc[b]["capacity"] for b in B ), 
             name='5.total loan respect bank cap')

m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                           for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               >= 0.1 * bank.iloc[b]["capacity"] for b in B),
             name='6.each bank spend 10% of cap')

m.addConstrs( (gp.quicksum( x[t,l,b] for b in B ) == gp.quicksum( z[t,l,p] for p in P )
            for t in T for l in L if loan.iloc[l]["loan_type"]==t ),
            name='7. if 1 loan to bank -> 1 loan to a banker')

m.addConstrs( ( x[t,l,b] + z[t,l,p] <= 1.5 + y[p,b] 
               for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B for p in P),
            name='8.Link z and y')

# Constraint: The value of investment loans cannot exceed 50% of total for each bank
m.addConstrs((gp.quicksum(loan.loc[l, "loan_amount"] * x[1, l, b] for l in L if loan.iloc[l]["loan_type"]==1) <=
              (gp.quicksum(0.5 * loan.loc[l, "loan_amount"] * x[t, l, b] for t in T for l in L if loan.iloc[l]["loan_type"]==t))
             for b in B),
             name="9.investment loan <= 50% percent per bank")

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 336 rows, 288 columns and 1323 nonzeros
Model fingerprint: 0x73342c79
Variable types: 0 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 30.0554637
Presolve removed 46 rows and 150 columns
Presolve time: 0.00s
Presolved: 290 rows, 138 columns, 1128 nonzeros
Variable types: 0 continuous, 138 integer (138 binary)

Root relaxation: objective 0.000000e+00, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

In [5]:
print([P,B,L,T])
print('Total value: ', m.objVal)
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

[range(0, 6), range(0, 3), range(0, 15), range(0, 2)]
Total value:  29.09465983448602
x[0,1,0] 1.0
x[0,6,1] 1.0
x[0,9,1] 1.0
x[0,14,2] 1.0
x[1,4,2] 1.0
y[0,2] 1.0
y[2,1] 1.0
y[3,0] 1.0
z[0,1,3] 1.0
z[0,6,2] 1.0
z[0,9,2] 1.0
z[0,14,0] 1.0
z[1,4,0] 1.0


---

## `Model 2 (Obj Function 1)` 

---

In [14]:
m = gp.Model()
#     m = gp.Model(env=env) # Google collab
m.Params.OutputFlag = 1

x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)

xr = m.addVars(T,L,B, name="xr", vtype = GRB.CONTINUOUS)
# aux var

r = m.addVar(name="r", lb=0, ub=24,vtype = GRB.CONTINUOUS)
# the rate bank can make profit from loan present without %

R = m.addVar(name="R", lb=0, ub=1, vtype = GRB.CONTINUOUS)
# constant based on r

r_obj_aux = m.addVar(name="Rr", vtype = GRB.CONTINUOUS)
R_exp_input_aux = m.addVar(name="R-exp-input", lb=-4, ub=0, vtype = GRB.CONTINUOUS)
test_R = m.addVar(name="test_R", vtype = GRB.CONTINUOUS)

# Objective
m.setObjective( (gp.quicksum( xr[t,l,b]*loan.iloc[l]["loan_amount"]
                    for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B) -
                gp.quicksum( dist[p][b]*y[p,b]
                    for b in B for p in P) -
                gp.quicksum( banker.iloc[p]["salary (in k)"]*y[p,b]*salary_rescale
                    for p in P for b in B)),
                GRB.MAXIMIZE)

## Constraints ##
# Loans # 
m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
               <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
            name='1.each loan <= 1 bank')

m.addConstrs( (gp.quicksum( z[t,l,p] for p in P) 
               <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
            name='2.each loan to <= 1 banker')

m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
               <= 1 for p in P),
            name='3.each banker <= 1 bank')

m.addConstrs( (gp.quicksum( z[t,l,p] * loan.iloc[l]["loan_amount"] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <=  banker.iloc[p]["capacity"] for p in P ),
            name='4.value of loans respect banker cap')

m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <= bank.iloc[b]["capacity"] for b in B ), 
            name='5.total loan respect bank cap')

m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               >= 0.11 * bank.iloc[b]["capacity"] for b in B),
            name='6.each bank spend 10% of cap')

m.addConstrs( (gp.quicksum( x[t,l,b] for b in B ) == 
               gp.quicksum( z[t,l,p] for p in P )
                for t in T for l in L if loan.iloc[l]["loan_type"]==t ),
            name='7. if 1 loan to bank -> 1 loan to a banker')

m.addConstrs( (x[t,l,b] + z[t,l,p] <= 1.5 + y[p,b] 
                for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B for p in P),
            name='8.Link z and y')

# Constraint: The value of investment loans cannot exceed 50% of total for each bank
m.addConstrs( (gp.quicksum(loan.loc[l, "loan_amount"] * x[1, l, b]
                for l in L if loan.iloc[l]["loan_type"]==1) <=
              (gp.quicksum(0.5 * loan.loc[l, "loan_amount"] * x[t, l, b] 
                for t in T for l in L if loan.iloc[l]["loan_type"]==t))
             for b in B),
             name="9.investment loan <= 50% percent per bank")

m.addConstrs( (x[t,l,b]*r_obj_aux == xr[t,l,b]
               for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B),
            name="11.object func aux var ")

# m.addConstr( R == (-(r/24)+1) )  # old 
# m.addGenConstrExp(z1,z2)   # z2 = exp(z1)

# m.addConstr( R_exp_input_aux == r*(-1/8))
# m.addGenConstrExp(R_exp_input_aux, R)   # R = exp(R_exp_input_aux)

m.addConstr( R == -(r/24)+1 )
m.addConstr( r_obj_aux == R*(r/100) )

m.setParam("NonConvex", 2)
m.setParam("MIPFocus", 3)
m.setParam("Cuts", 2)
m.optimize()

Set parameter NonConvex to value 2
Set parameter MIPFocus to value 3
Set parameter Cuts to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 337 rows, 383 columns and 1325 nonzeros
Model fingerprint: 0x160c7ea8
Model has 46 quadratic constraints
Variable types: 95 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [4e-02, 6e+02]
  QMatrix range    [1e-02, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [9e+00, 6e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 5e+03]
Presolve removed 46 rows and 197 columns
Presolve time: 0.00s
Presolved: 475 rows, 231 columns, 1543 nonzeros
Presolved model has 1 bilinear constraint(s)
Variable types: 93 continuous, 138 integer (138 binary)
Root relaxation presolve removed 48 rows and 46 columns
Root

In [15]:
print([P,B,L,T])
print('Total value: ', m.objVal)
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

[range(0, 6), range(0, 3), range(0, 15), range(0, 2)]
Total value:  138.96507644888462
x[0,1,1] 1.0
x[0,2,1] 1.0
x[0,3,2] 1.0
x[0,5,2] 1.0
x[0,7,2] 1.0
x[0,8,0] 1.0
x[0,9,0] 1.0
x[0,11,2] 1.0
x[0,12,1] 1.0
x[0,14,1] 1.0
x[1,0,2] 1.0
x[1,4,1] 1.0
x[1,10,1] 1.0
y[0,2] 1.0
y[1,2] 1.0
y[2,0] 1.0
y[3,1] 1.0
y[4,1] 1.0
y[5,1] 1.0
z[0,1,3] 1.0
z[0,2,5] 1.0
z[0,3,1] 1.0
z[0,5,0] 1.0
z[0,7,1] 1.0
z[0,8,2] 1.0
z[0,9,2] 1.0
z[0,11,0] 1.0
z[0,12,5] 1.0
z[0,14,4] 1.0
z[1,0,0] 1.0
z[1,4,4] 1.0
z[1,10,4] 1.0
xr[0,1,1] 0.06000000000000001
xr[0,2,1] 0.06000000000000001
xr[0,3,2] 0.06
xr[0,5,2] 0.06000000000000004
xr[0,7,2] 0.06
xr[0,8,0] 0.06000000000000004
xr[0,9,0] 0.06000000000000001
xr[0,11,2] 0.06000000000000001
xr[0,12,1] 0.06
xr[0,14,1] 0.06000000000000004
xr[1,0,2] 0.06000000000000001
xr[1,4,1] 0.06000000000000004
xr[1,10,1] 0.06000000000000001
r 12.038811011956389
R 0.4979703429290754
Rr 0.059999999999999984


---

## `Model 3 (Obj Function 2)` 

---

In [8]:
m = gp.Model()
# m = gp.Model(env=env) # Google collab
m.Params.OutputFlag = 1

x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)

xr = m.addVars(T,L,B, name="xr", vtype = GRB.CONTINUOUS)
# aux var

r = m.addVar(name="r", lb=0, ub=24,vtype = GRB.CONTINUOUS)
# the rate bank can make profit from loan present without %

R = m.addVar(name="R", lb=0, ub=1 , vtype = GRB.CONTINUOUS)
# constant based on r

r_obj_aux = m.addVar(name="Rr", vtype = GRB.CONTINUOUS)
n_assigned = m.addVar(name="n", vtype = GRB.CONTINUOUS)

# Objective
m.setObjective( (gp.quicksum( xr[t,l,b]*loan.iloc[l]["loan_amount"]
                    for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B) -
                gp.quicksum( dist[p][b]*y[p,b]
                    for b in B for p in P) -
                gp.quicksum( banker.iloc[p]["salary (in k)"]*y[p,b]*salary_rescale 
                    for p in P for b in B)) * n_assigned,
                GRB.MAXIMIZE)

m.addConstr( 1 == n_assigned*gp.quicksum(y[p,b] for b in B for p in P) )

## Constraints ##
m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
               <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
            name='1.each loan <= 1 bank')

m.addConstrs( (gp.quicksum( z[t,l,p] for p in P) 
               <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
            name='2.each loan to <= 1 banker')

m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
               <= 1 for p in P),
             name='3.each banker <= 1 bank')

m.addConstrs( (gp.quicksum( z[t,l,p] * loan.iloc[l]["loan_amount"] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <=  banker.iloc[p]["capacity"] for p in P ),
            name='4.value of loans respect banker cap')

m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               <= bank.iloc[b]["capacity"] for b in B ), 
            name='5.total loan respect bank cap')

m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
               >= 0.11 * bank.iloc[b]["capacity"] for b in B),
            name='6.each bank spend 10% of cap')

m.addConstrs( (gp.quicksum( x[t,l,b] for b in B ) == 
               gp.quicksum( z[t,l,p] for p in P )
                for t in T for l in L if loan.iloc[l]["loan_type"]==t ),
            name='7. if 1 loan to bank -> 1 loan to a banker')

m.addConstrs( (x[t,l,b] + z[t,l,p] <= 1.5 + y[p,b] 
                for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B for p in P),
            name='8.Link z and y')

# Constraint: The value of investment loans cannot exceed 50% of total for each bank
m.addConstrs( (gp.quicksum(loan.loc[l, "loan_amount"] * x[1, l, b]
                for l in L if loan.iloc[l]["loan_type"]==1) <=
              (gp.quicksum(0.5 * loan.loc[l, "loan_amount"] * x[t, l, b] 
                for t in T for l in L if loan.iloc[l]["loan_type"]==t))
             for b in B),
             name="9.investment loan <= 50% percent per bank")

m.addConstrs( (x[t,l,b]*r_obj_aux == xr[t,l,b]
               for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B),
            name="11.object func aux var ")

m.addConstr( R == -(r/24)+1 )
m.addConstr( r_obj_aux == R*(r/100) )

m.setParam("NonConvex", 2)
m.setParam("MIPFocus", 3)
m.setParam("Presolve", 2)
m.setParam("Cuts", 2)
m.optimize()

Set parameter NonConvex to value 2
Set parameter MIPFocus to value 3
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 337 rows, 382 columns and 1325 nonzeros
Model fingerprint: 0x609b05a9
Model has 63 quadratic objective terms
Model has 47 quadratic constraints
Variable types: 94 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [4e-02, 6e+02]
  QMatrix range    [1e-02, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+01, 1e+03]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 5e+03]
  QRHS range       [1e+00, 1e+00]
Presolve removed 46 rows and 195 columns
Presolve time: 0.01s
Presolved: 608 rows, 334 columns, 1961 nonzeros
Presolved model has 

In [9]:
print([P,B,L,T])
print('Total value: ', m.objVal)
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

[range(0, 6), range(0, 3), range(0, 15), range(0, 2)]
Total value:  37.68669519555216
x[0,1,0] 1.0
x[0,3,2] 1.0
x[0,5,0] 1.0
x[0,8,1] 1.0
x[0,11,0] 1.0
x[0,12,1] 1.0
x[0,14,2] 1.0
x[1,0,2] 1.0
x[1,10,0] 1.0
y[0,2] 1.0
y[4,0] 1.0
y[5,1] 1.0
z[0,1,4] 1.0
z[0,3,0] 1.0
z[0,5,4] 1.0
z[0,8,5] 1.0
z[0,11,4] 1.0
z[0,12,5] 1.0
z[0,14,0] 1.0
z[1,0,0] 1.0
z[1,10,4] 1.0
xr[0,1,0] 0.059999999999042423
xr[0,3,2] 0.059999999999042423
xr[0,5,0] 0.059999999999042423
xr[0,8,1] 0.059999999999042423
xr[0,11,0] 0.059999999999042423
xr[0,12,1] 0.059999999999042423
xr[0,14,2] 0.059999999999042423
xr[1,0,2] 0.059999999999042423
xr[1,10,0] 0.059999999999042423
r 12.000724442175478
R 0.49994350026561896
Rr 0.05999999999808481
n 0.3333333333333333


---

## `Model 2 with const. rate [1-24]` 

---

In [10]:
def model2_fixed_rate(r=r):
    print("r", r)
    m = gp.Model()
#     m = gp.Model(env=env) # Google collab
    m.Params.OutputFlag = 0

    x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
    y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
    z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)

    m.ModelSense = GRB.MAXIMIZE

    # Objective
    m.setObjective( gp.quicksum( x[t,l,b]*loan.iloc[l]["loan_amount"] * (-(r/24)+1) * (r/100)
                        for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B) -
                    gp.quicksum( dist[p][b]*y[p,b]
                        for b in B for p in P) -
                    gp.quicksum( banker.iloc[p]["salary (in k)"]*y[p,b]*salary_rescale 
                        for p in P for b in B),
                    GRB.MAXIMIZE)

    ## Constraints ##
    m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
                   <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
                name='1.each loan <= 1 bank')

    m.addConstrs( (gp.quicksum( z[t,l,p] for p in P) 
                   <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
                name='2.each loan to <= 1 banker')

    m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
                   <= 1 for p in P),
                name='3.each banker <= 1 bank')

    m.addConstrs( (gp.quicksum( z[t,l,p] * loan.iloc[l]["loan_amount"] 
                       for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   <=  banker.iloc[p]["capacity"] for p in P ),
                name='4.value of loans respect banker cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                       for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   <= bank.iloc[b]["capacity"] for b in B ), 
                name='5.total loan respect bank cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                       for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   >= 0.11 * bank.iloc[b]["capacity"] for b in B),
                name='6.each bank spend 10% of cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] for b in B ) == 
                   gp.quicksum( z[t,l,p] for p in P )
                    for t in T for l in L if loan.iloc[l]["loan_type"]==t ),
                name='7. if 1 loan to bank -> 1 loan to a banker')

    m.addConstrs( (x[t,l,b] + z[t,l,p] <= 1.5 + y[p,b] 
                    for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B for p in P),
                name='8.Link z and y')

    # Constraint: The value of investment loans cannot exceed 50% of total for each bank
    m.addConstrs( (gp.quicksum(loan.loc[l, "loan_amount"] * x[1, l, b]
                    for l in L if loan.iloc[l]["loan_type"]==1) <=
                  (gp.quicksum(0.5 * loan.loc[l, "loan_amount"] * x[t, l, b] 
                    for t in T for l in L if loan.iloc[l]["loan_type"]==t))
                 for b in B),
                 name="9.investment loan <= 50% percent per bank")

    m.setParam("NonConvex", 2)
    m.setParam("MIPFocus", 2)
    m.setParam("Cuts", 2)
    m.setParam("Presolve", 2)
    m.optimize()
    
    return m.objVal

In [11]:
ret_opt = [(i, model2_fixed_rate(i)) for i in range(25)]
ret_opt

r 0
0 -29.09465983448602
r 1
1 -8.492963742980958
r 2
2 11.777510046698913
r 3
3 35.13167671336558
r 4
4 56.02751004669892
r 5
5 74.46501004669891
r 6
6 90.64347088249731
r 7
7 105.08097088249731
r 8
8 116.89347088249731
r 9
9 126.40257644888456
r 10
10 133.38174311555122
r 11
11 137.56924311555125
r 12
12 138.96507644888456
r 13
13 137.56924311555122
r 14
14 133.38174311555125
r 15
15 126.40257644888456
r 16
16 116.89347088249731
r 17
17 105.08097088249731
r 18
18 90.64347088249731
r 19
19 74.46501004669895
r 20
20 56.0275100466989
r 21
21 35.13167671336558
r 22
22 11.777510046698938
r 23
23 -8.492963742980981
r 24
24 -29.09465983448602


[(0, -29.09465983448602),
 (1, -8.492963742980958),
 (2, 11.777510046698913),
 (3, 35.13167671336558),
 (4, 56.02751004669892),
 (5, 74.46501004669891),
 (6, 90.64347088249731),
 (7, 105.08097088249731),
 (8, 116.89347088249731),
 (9, 126.40257644888456),
 (10, 133.38174311555122),
 (11, 137.56924311555125),
 (12, 138.96507644888456),
 (13, 137.56924311555122),
 (14, 133.38174311555125),
 (15, 126.40257644888456),
 (16, 116.89347088249731),
 (17, 105.08097088249731),
 (18, 90.64347088249731),
 (19, 74.46501004669895),
 (20, 56.0275100466989),
 (21, 35.13167671336558),
 (22, 11.777510046698938),
 (23, -8.492963742980981),
 (24, -29.09465983448602)]

# Analysis

# `analysis_run.ipynb`

In [3]:
import gurobipy as gp
from gurobipy import GRB

import pandas as pd
import math

import time
from collections import defaultdict

## Actual Data

In [4]:
banker = pd.read_excel('.//banker_data//banker_data.xlsx')
bank = pd.read_csv('.//bank_loan_data//bank.csv')
loan = pd.read_csv('.//bank_loan_data//loan.csv')
loan_type = loan['loan_type'].unique()   # loan_type = [0,1] 
# type = ["living", "investment"]

# banker['salary (in k)'] = [i*1000 for i in banker['salary (in k)']]
# banker['capacity'] = [i*10000 for i in banker['capacity']]

# P = range(30) #range(round(len(banker)*0.1))
# B = range(2) #range(round(len(bank)*0.1))
# L = range(50) #range(round(len(loan)*0.02))

# T = range(len(loan_type))
# print([P,B,L,T])
        
# # yoinked from ass1 
# dist = [ [0]*len(B) for i in P]
# for p in P:
#      for b in B:
#         # LA,LONG for banker is column 1,2; for banks LONG,LA is 1:2
#         dist[p][b] = math.dist(banker.iloc[p, 1:3],bank.iloc[b, 2:0:-1])


# print("bank\n", bank.head())
# print("loan\n",loan.head())
# print("banker\n",banker.head())

## Dummy Data

In [38]:
P = range(6)
B = range(3)
L = range(15)
T = range(len(loan_type))
print([P,B,L,T])

banker = banker.iloc[0:6]
bank = bank.iloc[0:3]
loan = loan.iloc[0:15]

banker.loc[0, "capacity"] = 750
banker.loc[1,"capacity"] = 250
banker.loc[2, "capacity"] = 500
banker.loc[3,"capacity"] = 250
banker.loc[4, "capacity"] = 750
banker.loc[5,"capacity"] = 1000
# banker.loc[6, "capacity"] = 500
# banker.loc[7,"capacity"] = 250
# banker.loc[8, "capacity"] = 500
# banker.loc[9,"capacity"] = 500

bank.loc[0,"capacity"] = 1000
bank.loc[1,"capacity"] = 5000
bank.loc[2,"capacity"] = 3500

loan.loc[0,"loan_amount"] = 300
loan.loc[0,"loan_type"] = 1
loan.loc[1,"loan_amount"] = 200
loan.loc[1,"loan_type"] = 0
loan.loc[2,"loan_amount"] = 400
loan.loc[2,"loan_type"] = 0
loan.loc[3,"loan_amount"] = 100
loan.loc[3,"loan_type"] = 0
loan.loc[4,"loan_amount"] = 300
loan.loc[4,"loan_type"] = 1
loan.loc[5,"loan_amount"] = 200
loan.loc[5,"loan_type"] = 0
loan.loc[6,"loan_amount"] = 400
loan.loc[6,"loan_type"] = 0
loan.loc[7,"loan_amount"] = 100
loan.loc[7,"loan_type"] = 0
loan.loc[8,"loan_amount"] = 400
loan.loc[8,"loan_type"] = 0
loan.loc[9,"loan_amount"] = 100
loan.loc[9,"loan_type"] = 0
loan.loc[10,"loan_amount"] = 150
loan.loc[10,"loan_type"] = 1
loan.loc[11,"loan_amount"] = 200
loan.loc[11,"loan_type"] = 0
loan.loc[12,"loan_amount"] = 600
loan.loc[12,"loan_type"] = 0
loan.loc[13,"loan_amount"] = 100
loan.loc[13,"loan_type"] = 1
loan.loc[14,"loan_amount"] = 300
loan.loc[14,"loan_type"] = 0
# loan.loc[15,"loan_amount"] = 200
# loan.loc[15,"loan_type"] = 0
# loan.loc[16,"loan_amount"] = 400
# loan.loc[16,"loan_type"] = 0
# loan.loc[17,"loan_amount"] = 100
# loan.loc[17,"loan_type"] = 0
# loan.loc[18,"loan_amount"] = 400
# loan.loc[18,"loan_type"] = 0
# loan.loc[19,"loan_amount"] = 200
# loan.loc[19,"loan_type"] = 1

        
# Dummy distance between banker and branch
dist = [[5,8,1], [6,9,2], [1,2,4], [1,1,1], [4,3,5], [6,2,4]]#, [7,1,11], [8,2,3], [6,2,1], [2,3,2]]

print("bank\n", bank.head())
print("loan\n",loan.head())
print("banker\n",banker.head())

[range(0, 6), range(0, 3), range(0, 15), range(0, 2)]
bank
                                             location        long        lat   
0  Westfield Fountain Gate, 352 Princes Highway, ...  145.304441 -38.018206  \
1               330 Collins St, Melbourne, VIC, 3000  144.963217 -37.816309   
2               460 Collins St, Melbourne, VIC, 3000  144.958852 -37.817673   

   capacity  
0    1000.0  
1    5000.0  
2    3500.0  
loan
    loan_amount  loan_type
0        300.0          1
1        200.0          0
2        400.0          0
3        100.0          0
4        300.0          1
banker
    banker_id   Latitude   Longitude  tenure (in years)  capacity  v1_rand_0_1   
0          1 -37.695528  143.951218           8.183687       750     0.863803  \
1          2 -37.551700  144.405921           8.184793       250     0.539807   
2          3 -37.537275  144.841580           6.404775       500     0.934031   
3          4 -37.563307  144.933624           4.616158       250     0.88

In [39]:
assigned_loans = []
assigned_bankers = []
targets = []
branch = []
branch_dict = defaultdict()

In [40]:
for target in [i/100 for i in range(1, 41)]:
    start_time = time.time()
    m = gp.Model()
    m.Params.LogToConsole = 0
#     m.Params.OutputFlag = 1

    M = 100000
    x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
    y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
    z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)

    # Objective
    m.setObjective(gp.quicksum( dist[p][b]*y[p,b] for b in B for p in P) +
                    gp.quicksum( banker.iloc[p]["salary (in k)"]*y[p,b] for p in P for b in B),
                   GRB.MINIMIZE )

    ## Constraints ##
    m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
                   <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
                 name='1.each loan <= 1 bank')

    m.addConstrs( (gp.quicksum( z[t,l,p] for p in P) 
                   <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
                 name='2.each loan to <= 1 banker')

    m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
                   <= 1.5 for p in P),
                 name='3.each banker <= 1 bank')

    m.addConstrs( (gp.quicksum( z[t,l,p] * loan.iloc[l]["loan_amount"] 
                               for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   <=  banker.iloc[p]["capacity"] for p in P ),
                 name='4.value of loans respect banker cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                               for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   <= bank.iloc[b]["capacity"] for b in B ), 
                 name='5.total loan respect bank cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                               for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   >= target * bank.iloc[b]["capacity"] for b in B),
                 name='6.each bank spend 10% of cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] for b in B ) == gp.quicksum( z[t,l,p] for p in P )
                for t in T for l in L if loan.iloc[l]["loan_type"]==t ),
                name='7. if 1 loan to bank -> 1 loan to a banker')

    m.addConstrs( ( x[t,l,b] + z[t,l,p] <= 1.5 + y[p,b] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B for p in P),
                name='8.Link z and y')

    m.addConstrs((gp.quicksum(loan.loc[l, "loan_amount"] * x[1, l, b] for l in L if loan.iloc[l]["loan_type"]==1) <=
                  (gp.quicksum(0.5 * loan.loc[l, "loan_amount"] * x[t, l, b] for t in T for l in L if loan.iloc[l]["loan_type"]==t))
                 for b in B),
                 name="9.investment loan <= 50% percent per bank")

    print('optimizing...')
    m.optimize()

    for v in m.getVars():
        if v.x > 1e-6:
            if v.varName[0] == 'z':
                assigned_bankers.append(int(v.varName[2:-1].split(',')[2])+1)
                assigned_loans.append(int(v.varName[2:-1].split(',')[1]))
                targets.append(target)
                branch.append(branch_dict[int(v.varName[2:-1].split(',')[2])+1])
            if v.varName[0] == 'y':
                branch_dict[int(v.varName[2:-1].split(',')[0])+1] = int(v.varName[2:-1].split(',')[1])
    print(f"target {target} computed time taken {(time.time() - start_time)/60:}")

optimizing...
target 0.01 computed time taken 0.023477296034495037
optimizing...
target 0.02 computed time taken 0.024981717268625896
optimizing...
target 0.03 computed time taken 0.03667974869410197
optimizing...
target 0.04 computed time taken 0.028430430094401042
optimizing...
target 0.05 computed time taken 0.02457118034362793
optimizing...
target 0.06 computed time taken 0.027120681603749593
optimizing...
target 0.07 computed time taken 0.030775352319081625
optimizing...
target 0.08 computed time taken 0.02061848243077596
optimizing...
target 0.09 computed time taken 0.02765376170476278
optimizing...
target 0.1 computed time taken 0.02712634801864624
optimizing...
target 0.11 computed time taken 0.03921931982040405
optimizing...
target 0.12 computed time taken 0.0237336794535319
optimizing...
target 0.13 computed time taken 0.03274633089701335
optimizing...
target 0.14 computed time taken 0.031910812854766844
optimizing...
target 0.15 computed time taken 0.02891063690185547
optimi

AttributeError: Unable to retrieve attribute 'x'

In [41]:
assignment_df = pd.DataFrame({'banker_id':assigned_bankers, 
                              'branch_id': branch,
                              'loan_id':assigned_loans, 
                              'target':targets})

In [42]:
assignment_df = pd.merge(assignment_df, banker[['banker_id', 'capacity']], how='left', on='banker_id')

In [43]:
loan2 = loan.reset_index(names='loan_id')
bank2 = bank.reset_index(names='branch_id')

In [44]:
assignment_df = pd.merge(assignment_df, loan2, how='left', on='loan_id')
assignment_df = pd.merge(assignment_df, bank2[['branch_id','location']], how='left', on='branch_id')

In [45]:
assignment_df

,banker_id,branch_id,loan_id,target,capacity,loan_amount,loan_type,location
0,6,1,1,0.01,1000,200.0,0,"330 Collins St, Melbourne, VIC, 3000"
1,3,0,2,0.01,500,400.0,0,"Westfield Fountain Gate, 352 Princes Highway, ..."
2,4,2,3,0.01,250,100.0,0,"460 Collins St, Melbourne, VIC, 3000"
3,6,1,6,0.01,1000,400.0,0,"330 Collins St, Melbourne, VIC, 3000"
4,6,1,1,0.02,1000,200.0,0,"330 Collins St, Melbourne, VIC, 3000"
...,...,...,...,...,...,...,...,...
260,3,2,14,0.34,500,300.0,0,"460 Collins St, Melbourne, VIC, 3000"
261,5,1,0,0.34,750,300.0,1,"330 Collins St, Melbourne, VIC, 3000"
262,1,2,4,0.34,750,300.0,1,"460 Collins St, Melbourne, VIC, 3000"
263,2,0,10,0.34,250,150.0,1,"Westfield Fountain Gate, 352 Princes Highway, ..."


In [49]:
cols = ['banker_id', 'branch_id', 'capacity', 'target', 'loan_amount']
grouped_df = assignment_df[cols].groupby(['banker_id', 'branch_id','capacity', 'target']).sum().reset_index()

In [50]:
grouped_df['excess_capacity'] = grouped_df['capacity'] - grouped_df['loan_amount']

In [51]:
grouped_df[grouped_df.target == 0.01]

,banker_id,branch_id,capacity,target,loan_amount,excess_capacity
30,3,0,500,0.01,400.0,100.0
76,4,2,250,0.01,100.0,150.0
98,6,1,1000,0.01,600.0,400.0


In [53]:
grouped_df.to_csv('model_1_results.csv')

In [74]:
assignment_df[assignment_df.target==0.24]

,banker_id,loan_id,target,capacity,loan_amount,loan_type
87,2,1,0.24,250,200.0,0
88,3,3,0.24,500,100.0,0
89,1,6,0.24,750,400.0,0
90,4,7,0.24,250,100.0,0
91,3,8,0.24,500,400.0,0
92,1,0,0.24,750,300.0,1


In [ ]:
grouped_df

# Data Generation & Compilation Section

# `get_branch_data_dev`

In [36]:
!pip install selenium

In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import re
import pandas as pd

In [2]:
links = []

for i in range(1,12):
    driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')
    if i == 1:
        driver.get('https://nab.banklocationmaps.com.au/en/branches/aus/victoria/')
    else:
        driver.get(f'https://nab.banklocationmaps.com.au/en/branches/aus/victoria?page={i}')
    
    directions = driver.find_elements(By.XPATH, "//a[@class='btn-sm btn-outline-secondary btn-shadow btn-block']")
    for d in directions:
        links.append(d.get_attribute("href"))

    driver.close()

/var/folders/95/76mmy6pn2kvccmkyykdt6tvw0000gn/T/ipykernel_29321/1366198960.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')


In [15]:
all_branch

{'Westfield Fountain Gate, 352 Princes Highway, Westfield Fountain Gate, Narre Warren, VIC, 3805': {'long': 145.304441,
  'lat': -38.018206,
  'OH': ['Monday-Thursday:9.30am-4.00pm', 'Friday:9.30am-5.00pm']}}

In [10]:
'a'.replace('a', 'b')

'b'

In [17]:
all_branch = {}
def get_longlat(locs, text):
    branch = ''
    for line in text.split('\n'):
        if '"streetAddress"' in line:
            branch = line.split(': ')[1][1:-2]
            locs[branch] = {'long':0, 'lat': 0}
        if '"longitude"' in line:
            locs[branch]['long'] = float(line.split(': ')[1])
        elif '"latitude"' in line:
            locs[branch]['lat'] = float(line.split(': ')[1][:-1])
        if 'Monday' in line:
            locs[branch]['OH'] = [s.replace('</td><td>', ':') for s in line[15:-10].split('</td></tr><tr><td>')]
    return locs

In [22]:
vic_branches = [l for l in links if l[-9:] == 'locations']
for link in vic_branches[60:]:
    print(link)
    driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')
    driver.get(link)
    geo = driver.page_source
    all_branch = get_longlat(all_branch, geo)
    driver.close()
if len(all_branch) != len(vic_branches):
    print(f"""Warning - Number of extracted NAB Victorian branches {len(all_branch)}, 
          expected {len(vic_branches)} branches!""")

https://nab.banklocationmaps.com.au/en/branch/270562-nab-branch-cranbourne-park-shopping-centre-high-st-cranbourne-park-shopping-centre?directions=1#locations


/var/folders/95/76mmy6pn2kvccmkyykdt6tvw0000gn/T/ipykernel_29321/331689300.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')


https://nab.banklocationmaps.com.au/en/branch/89370-nab-branch-100-exhibition-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/89517-nab-branch-183-manifold-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/97449-nab-branch-southland-shopping-centre-westfield-southland-1239-nepean-highway?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/89423-nab-branch-120-upper-heidelberg-road?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/89612-nab-branch-102-106-main-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/89648-nab-branch-21-brook-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90210-nab-branch-79-81-malop-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90244-nab-branch-3-ordish-road?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/98245-nab-branch-waurn-ponds-shopping-centre-173-199-pioneer-road-waurn-ponds-shopp

https://nab.banklocationmaps.com.au/en/branch/90042-nab-branch-50-ranelagh-dr?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90294-nab-branch-76-high-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/95783-nab-branch-67-high-street?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/97351-nab-branch-drysdale-village-shopping-centre-16-wyndham-st-drysdale-village-shopping-centre?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/89429-nab-branch-185-high-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/89654-nab-branch-158-scott-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90037-nab-branch-40-station-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90179-nab-branch-100-exhibition-street?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90180-nab-branch-the-concourse-700-bourke-st-the-concourse?directions=1#locations
ht

https://nab.banklocationmaps.com.au/en/branch/90300-nab-branch-16-gilbert-street?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/90305-nab-branch-299-301-esplanade?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/97169-nab-branch-midtown-plaza-186-swanston-street-midtown-plaza?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/97390-nab-branch-sunshine-marketplace-80-harvester-road-sunshine-marketplace?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/272761-nab-branch-2-eaton-street?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/278117-nab-branch-2-eaton-st?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/338185-nab-branch-445-toorak-rd?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/338191-nab-branch-800-bourke-street?directions=1#locations
https://nab.banklocationmaps.com.au/en/branch/338194-nab-branch-104-main-st?directions=1#locations
https:/

In [23]:
# Check for duplicated branch names
len(set([re.search("nab-branch-(.*)?directions=1", i).group(0) for i in vic_branches]))

180

In [ ]:
# sorted([ i for i in all_branch.keys()])

In [24]:
# Get duplicates based on long lat locations
dups = []
for k1, v1 in all_branch.items():
    for k2, v2 in all_branch.items():
        if k2 != k1 and v2 == v1 and k1 not in dups:
            dups.append(k2)

In [25]:
sorted(dups)

['100 Exhibition Street, Melbourne, VIC, 3000',
 '107 Main St, Mornington, VIC, 3931',
 '224-228 Smith Street, Collingwood, VIC, 3066',
 '293 Spring St, Reservoir, VIC, 3073',
 '406 Sydney Rd, Coburg, VIC, 3058',
 '566 Burke Road, Camberwell, VIC, 3124',
 '82-88 Franklin St, Traralgon, VIC, 3844',
 'Melbourne NAB House, 500 Bourke Street, Melbourne NAB House, Melbourne, VIC, 3000',
 'The Concourse, 700 Bourke St, The Concourse, Docklands, VIC, 3008']

In [26]:
# Remove duplicate branches
final_branch_set = {k: all_branch[k] for k in all_branch.keys() if k not in dups}

In [27]:
len(final_branch_set)

171

In [28]:
pd.DataFrame.from_dict(final_branch_set, orient='index')\
  .reset_index(names='branch')\
  .to_csv('bank_loan_data/branch_locations.csv', index=False)

# `get_bank_and_loan_data.ipynb`

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
bank=pd.read_csv('bank_loan_data/branch_locations.csv')
bank['ID'] = bank.index
bank=bank[['ID','branch','long','lat']]
bank.rename(columns={'branch':'location'},inplace=True)
bank

,ID,location,long,lat
0,0,"Westfield Fountain Gate, 352 Princes Highway, ...",145.304441,-38.018206
1,1,"330 Collins St, Melbourne, VIC, 3000",144.963217,-37.816309
2,2,"460 Collins St, Melbourne, VIC, 3000",144.958852,-37.817673
3,3,"288 Lygon Street, Carlton, VIC, 3053",144.967263,-37.800216
4,4,"22 Watton St, Werribee, VIC, 3030",144.661003,-37.901344
...,...,...,...,...
164,164,"40 Federal St, Rainbow, VIC, 3424",141.994995,-35.898205
165,165,"16 Gilbert Street, Torquay, VIC, 3228",144.324342,-38.332092
166,166,"299-301 Esplanade, Lakes Entrance, VIC, 3909",147.983521,-37.881281
167,167,"Midtown Plaza, 186 Swanston Street, Midtown Pl...",144.965472,-37.813131


In [3]:
location=bank['location'].tolist()
location

['Westfield Fountain Gate, 352 Princes Highway, Westfield Fountain Gate, Narre Warren, VIC, 3805',
 '330 Collins St, Melbourne, VIC, 3000',
 '460 Collins St, Melbourne, VIC, 3000',
 '288 Lygon Street, Carlton, VIC, 3053',
 '22 Watton St, Werribee, VIC, 3030',
 'Northland Shopping Centre, 2-50 Murray Road, Northland Shopping Centre, Preston, VIC, 3072',
 '260 Clarendon St, South Melbourne, VIC, 3205',
 '415 High Street, Preston, VIC, 3072',
 'Brimbank Plaza Shopping Centre, 28 Neale Road, Corner Station Road, Brimbank Plaza Shopping Centre, Deer Park, VIC, 3023',
 'Mountain Gate Shopping Centre, 854 Burwood Highway, Mountain Gate Shopping Centre, Ferntree Gully, VIC, 3156',
 'Eastland Shopping Centre, 171-175 Maroondah Highway, Eastland Shopping Centre, Ringwood, VIC, 3134',
 '637 Glenferrie Rd, Hawthorn, VIC, 3122',
 'Westfield Knox, 425 Burwood Highway, Westfield Knox, Wantirna South, VIC, 3152',
 '224-228 Smith St, Collingwood, VIC, 3066',
 'Stud Park Shopping Centre, Cnr Stud & Fulh

In [4]:
bank_suburb=[]
for l in location:
    ll=l.split(', ')
    bank_suburb.append(ll[-3])
bank_suburb

['Narre Warren',
 'Melbourne',
 'Melbourne',
 'Carlton',
 'Werribee',
 'Preston',
 'South Melbourne',
 'Preston',
 'Deer Park',
 'Ferntree Gully',
 'Ringwood',
 'Hawthorn',
 'Wantirna South',
 'Collingwood',
 'Rowville',
 'Richmond',
 'Melbourne',
 'Doncaster',
 'Colac',
 'Brighton',
 'Whittlesea',
 'Chadstone',
 'Camberwell',
 'Balwyn',
 'Docklands',
 'Ballarat',
 'Reservoir',
 'Docklands',
 'Greensborough',
 'Box Hill',
 'Seaford',
 'Keysborough',
 'Croydon',
 'Rosebud',
 'Ocean Grove',
 'Thomastown',
 'Kew',
 'Berwick',
 'Belmont',
 'Wodonga',
 'North Melbourne',
 'Wangaratta',
 'Melbourne',
 'Niddrie',
 'Doncaster East',
 'Swan Hill',
 'Newcomb',
 'Forest Hill',
 'Port Melbourne',
 'Healesville',
 'Geelong West',
 'Lilydale',
 'Cheltenham',
 'Mornington',
 'Moe',
 'Wheelers Hill',
 'Maribyrnong',
 'Bacchus Marsh',
 'Elsternwick',
 'Cranbourne',
 'Melbourne',
 'Camperdown',
 'Ivanhoe',
 'Pakenham',
 'Sunbury',
 'Geelong',
 'Dandenong South',
 'Waurn Ponds',
 'Horsham',
 'Clayton',
 

In [5]:
len(bank_suburb)

169

In [6]:
house=pd.read_excel('bank_loan_data/PSV_SUBURB_HOUSE_1stQrt_2022.xlsx',index_col='SUBURB')

In [7]:
for i in range(len(bank_suburb)):
    s = bank_suburb[i]
    if s.upper() not in house.index.tolist():
        print(i,s)

1 Melbourne
2 Melbourne
16 Melbourne
24 Docklands
25 Ballarat
27 Docklands
42 Melbourne
60 Melbourne
66 Dandenong South
74 Melbourne
80 Melbourne
96 Geelong North
103 Narre Warre South
119 Ballarat
131 Moorabbin Airport
155 Balmoral
167 Melbourne


In [8]:
dd=dict(Counter(bank_suburb))
sorted(list(dd.items()))

[('Airport West', 1),
 ('Alexandra', 1),
 ('Ararat', 1),
 ('Bacchus Marsh', 1),
 ('Bairnsdale', 1),
 ('Ballarat', 2),
 ('Balmoral', 1),
 ('Balwyn', 1),
 ('Bayswater', 1),
 ('Belmont', 1),
 ('Benalla', 1),
 ('Bendigo', 1),
 ('Bentleigh', 1),
 ('Berwick', 1),
 ('Box Hill', 1),
 ('Bright', 1),
 ('Brighton', 1),
 ('Broadmeadows', 1),
 ('Camberwell', 1),
 ('Camperdown', 1),
 ('Carlton', 1),
 ('Carnegie', 1),
 ('Caroline Springs', 1),
 ('Castlemaine', 1),
 ('Chadstone', 1),
 ('Cheltenham', 1),
 ('Clayton', 1),
 ('Cobden', 1),
 ('Cobram', 1),
 ('Coburg', 1),
 ('Cohuna', 1),
 ('Colac', 1),
 ('Collingwood', 1),
 ('Cowes', 1),
 ('Craigieburn', 1),
 ('Cranbourne', 1),
 ('Croydon', 1),
 ('Dandenong', 1),
 ('Dandenong South', 1),
 ('Deer Park', 1),
 ('Docklands', 2),
 ('Doncaster', 1),
 ('Doncaster East', 1),
 ('Dromana', 1),
 ('Drysdale', 1),
 ('Echuca', 1),
 ('Edenhope', 1),
 ('Elsternwick', 1),
 ('Emerald', 1),
 ('Epping', 1),
 ('Ferntree Gully', 1),
 ('Forest Hill', 1),
 ('Frankston', 1),
 ('Ge

## GENERATE LOAN DATA FROM HOUSE DATA

In [9]:
#read house data
house=pd.read_excel('bank_loan_data/PSV_SUBURB_HOUSE_1stQrt_2022.xlsx',index_col='SUBURB')
house

,PRICE,AMOUNT
SUBURB,,
ABBOTSFORD,1590000,18
ABERFELDIE,1892500,14
AINTREE,800000,37
AIREYS INLET,1750000,4
AIRPORT WEST,928000,27
...,...,...
YARRAM,452500,14
YARRAVILLE,1355000,48
YARRAWONGA,732000,40


In [10]:
# generate loans based on house prices and no. of sales across suburbs in VIC
np.random.seed(2023)
loan_amount = []
loan_type = []
for i,row in house.iterrows():
    p = row['PRICE']
    n = row['AMOUNT']
    loan_amount+=list(np.round(np.random.normal(p, 100, n),0))
    loan_type+=list(np.random.randint(0,2,n))
    
        

In [11]:
loan_amount[:15]

[1590071.0,
 1589968.0,
 1589900.0,
 1590024.0,
 1589990.0,
 1589886.0,
 1590265.0,
 1590144.0,
 1590010.0,
 1589688.0,
 1589892.0,
 1589967.0,
 1589896.0,
 1589957.0,
 1589897.0]

In [12]:
print('total number:',len(loan_amount),' total amount:',sum(loan_amount))
print('total number:',len(loan_type),' total amount:',sum(loan_type))

total number: 21095  total amount: 19967929602.0
total number: 21095  total amount: 10629


In [13]:
#total amount of loans per bank
sum(loan_amount)/len(bank)

118153429.59763314

In [14]:
loan_df=pd.DataFrame({'loan_amount':loan_amount,'loan_type':loan_type})
loan_df

,loan_amount,loan_type
0,1590071.0,1
1,1589968.0,1
2,1589900.0,0
3,1590024.0,1
4,1589990.0,1
...,...,...
21090,609948.0,1
21091,562556.0,1
21092,562592.0,1
21093,562554.0,1


In [15]:
loan_df.to_csv('bank_loan_data/loan.csv',sep=',',index=False,header=True)

# GENERATING THE CAPACITY FOR EACH BANK

## 1. randomly generate (all banks have a similar capcity) 

In [16]:
capacity = []
np.random.seed(2023)
# cap = mean of total loans * some random number between 0.95-1
capacity=list(np.round(np.random.randint(95,100,len(bank))/100*sum(loan_amount)/(len(bank))))
capacity[:10]

[113427292.0,
 113427292.0,
 116971895.0,
 115790361.0,
 116971895.0,
 116971895.0,
 112245758.0,
 113427292.0,
 112245758.0,
 114608827.0]

In [17]:
print('no. of bank:',len(capacity),' total capacity:',sum(capacity))

no. of bank: 169  total capacity: 19399611585.0


## 2. weighted randomly generate

In [18]:
# assume a bank in a suburb with a higher house price needs a higher capcity
bank_suburb_price=[]
for i in range(len(bank_suburb)):
    s = bank_suburb[i]
    if s.upper() in house.index.tolist():
        #print(i,s)
        bank_suburb_price.append(house.loc[s.upper()]['PRICE'])
    # if the suburb that a bank located not in the house data
    elif s.upper()=='BALLARAT':
        bank_suburb_price.append(house.loc['BALLARAT CENTRAL']['PRICE'])
    elif s.upper()=='BALMORAL':
        bank_suburb_price.append(house.loc['COLERAINE']['PRICE'])
    elif s.upper()=='DANDENONG SOUTH':
        bank_suburb_price.append((house.loc['DANDENONG']['PRICE']+house.loc['DANDENONG NORTH']['PRICE'])/2)
    elif s.upper()=='DOCKLANDS':
        bank_suburb_price.append(550000)
    elif s.upper()=='GEELONG NORTH':
        bank_suburb_price.append((house.loc['GEELONG']['PRICE']+house.loc['GEELONG WEST']['PRICE'])/2)
    elif s.upper()=='MOORABBIN AIRPORT':
        bank_suburb_price.append(house.loc['MOORABBIN']['PRICE'])
    elif s=='Melbourne':
        bank_suburb_price.append(1116000)
    elif s=='Narre Warre South':
        bank_suburb_price.append(house.loc['NARRE WARREN SOUTH']['PRICE'])
bank_suburb_price

[771500,
 1116000,
 1116000,
 1867500,
 628000,
 1315000,
 1842500,
 1315000,
 646000,
 911800,
 1046500,
 2480000,
 1372500,
 1289000,
 1102000,
 1380100,
 1116000,
 1550000,
 500000,
 3380000,
 800000,
 1192500,
 2650000,
 2833000,
 550000,
 760000,
 1005000,
 550000,
 990000,
 1810000,
 940000,
 925000,
 980000,
 852000,
 1127500,
 725000,
 2737500,
 903100,
 795000,
 499500,
 1310000,
 530000,
 1116000,
 1376500,
 1567500,
 359000,
 610000,
 1245000,
 1567500,
 820000,
 1060000,
 860000,
 1493500,
 1225000,
 360000,
 1486500,
 1170000,
 662500,
 2337500,
 677500,
 1116000,
 440000,
 2015000,
 662000,
 675000,
 855000,
 734300.0,
 817500,
 388800,
 1100000,
 1650500,
 310000,
 426500,
 4377500,
 1116000,
 700000,
 4000000,
 1600000,
 720000,
 965000,
 1116000,
 654000,
 597500,
 1935000,
 1745000,
 585000,
 673500,
 435000,
 989500,
 525000,
 438000,
 787000,
 500000,
 732000,
 830000,
 670000,
 957500.0,
 875000,
 140000,
 1740000,
 735000,
 592500,
 886500,
 818000,
 285000,
 7650

In [19]:
#check this list's length
len(bank_suburb_price)==len(bank)

True

In [20]:
bank_weight=bank_suburb_price/sum(bank_suburb_price)
np.random.seed(2023)
# cap = weight * total amount of loan * some random number between 0.95-1
capacity2=list(np.round(bank_weight*np.random.randint(95,100,len(bank))/100*sum(loan_amount)))
capacity2[:10]

[91908072.0,
 132948034.0,
 137102661.0,
 227108391.0,
 77150960.0,
 161550178.0,
 217208889.0,
 156654718.0,
 76155735.0,
 109753358.0]

In [21]:
print('no. of bank:',len(capacity2),' total capacity:',sum(capacity2))

no. of bank: 169  total capacity: 19391134963.0


In [22]:
# chose capacity2
bank_df=pd.DataFrame({'location':bank['location'],'long':bank['long'],'lat':bank['lat'],'capacity':capacity2})
bank_df

,location,long,lat,capacity
0,"Westfield Fountain Gate, 352 Princes Highway, ...",145.304441,-38.018206,91908072.0
1,"330 Collins St, Melbourne, VIC, 3000",144.963217,-37.816309,132948034.0
2,"460 Collins St, Melbourne, VIC, 3000",144.958852,-37.817673,137102661.0
3,"288 Lygon Street, Carlton, VIC, 3053",144.967263,-37.800216,227108391.0
4,"22 Watton St, Werribee, VIC, 3030",144.661003,-37.901344,77150960.0
...,...,...,...,...
164,"40 Federal St, Rainbow, VIC, 3424",141.994995,-35.898205,16799059.0
165,"16 Gilbert Street, Torquay, VIC, 3228",144.324342,-38.332092,162499488.0
166,"299-301 Esplanade, Lakes Entrance, VIC, 3909",147.983521,-37.881281,66277951.0
167,"Midtown Plaza, 186 Swanston Street, Midtown Pl...",144.965472,-37.813131,137102661.0


In [23]:
bank_df.to_csv('bank_loan_data/bank.csv',sep=',',index=False,header=True)

# `OPT_A2.ipynb`

In [17]:
import folium
import random
import pandas as pd
# from google.colab import files

# Define the range of latitude and longitude for Victoria
Mel_lat_min, Mel_lat_max =  -37.8136, -37.5108
Mel_lon_min, Mel_lon_max = 143.8503, 145.5125 

Geelong_lat_min, Geelong_lat_max = -38.3129, -38.0659
Geelong_lon_min, Geelong_lon_max = 144.2804, 144.6232

Wyndham_lat_min, Wyndham_lat_max = -37.999, -37.833
Wyndham_lon_min, Wyndham_lon_max = 144.553, 144.825

Bendigo_lat_min, Bendigo_lat_max = -37.041, -35.858
Bendigo_lon_min, Bendigo_lon_max = 143.398, 144.643

Ballarat_lat_min, Ballarat_lat_max = -37.7, -37.5
Ballarat_lon_min, Ballarat_lon_max = 143.5, 143.8

Shepparton_lat_min, Shepparton_lat_max = -36.39, -36.03
Shepparton_lon_min, Shepparton_lon_max = 145.34, 145.57

Casey_lat_min, Casey_lat_max = -38.3587, -37.9058
Casey_lon_min, Casey_lon_max = 145.1857, 145.6954

In [18]:
# Define the number of random locations to generate
Melb_locations = 43
Geelong_loc = 68
Wyndham_loc = 73
Casey_loc = 91
Bendigo_loc = 30
Ballarat_loc = 30
Shepparton_loc = 15

# Generate the random locations
locations = []
for i in range(Melb_locations):
    lat = random.uniform(Mel_lat_min, Mel_lat_max)
    lon = random.uniform(Mel_lon_min, Mel_lon_max)
    locations.append((lat, lon))

for i in range(Geelong_loc):
    lat = random.uniform(Geelong_lat_min, Geelong_lat_max)
    lon = random.uniform(Geelong_lon_min, Geelong_lon_max)
    locations.append((lat, lon))

for i in range(Wyndham_loc):
    lat = random.uniform(Wyndham_lat_min, Wyndham_lat_max)
    lon = random.uniform(Wyndham_lon_min, Wyndham_lon_max)
    locations.append((lat, lon))

for i in range(Casey_loc):
    lat = random.uniform(Casey_lat_min, Casey_lat_max)
    lon = random.uniform(Casey_lon_min, Casey_lon_max)
    locations.append((lat, lon))

for i in range(Bendigo_loc):
    lat = random.uniform(Bendigo_lat_min, Bendigo_lat_max)
    lon = random.uniform(Bendigo_lon_min, Bendigo_lon_max)
    locations.append((lat, lon))

for i in range(Ballarat_loc):
    lat = random.uniform(Ballarat_lat_min, Ballarat_lat_max)
    lon = random.uniform(Ballarat_lon_min, Ballarat_lon_max)
    locations.append((lat, lon))

for i in range(Shepparton_loc):
    lat = random.uniform(Shepparton_lat_min, Shepparton_lat_max)
    lon = random.uniform(Shepparton_lon_min, Shepparton_lon_max)
    locations.append((lat, lon))

# Create a map centered on Melbourne
map_victoria = folium.Map(location=[-37.8136, 144.9631], zoom_start=8)

# Add a marker for each location
for location in locations:
    folium.Marker(location).add_to(map_victoria)

# Show the map
map_victoria

In [19]:
df = pd.DataFrame(locations, columns=['Latitude', 'Longitude'])
pd.set_option('display.max_rows', None)

# print(df)
# Export the dataframe to an Excel file
# df.to_excel('vic_locations.xlsx', index=False)
# files.download('vic_locations.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

OPT_A2 was ran on Google Colab and the rest of the banker data is generated using Excel functions. The details of how and what data are generated are included in the report.